In [1]:
import os
import re
import pandas as pd
from typing import List, Dict, Union

# 이전 단계에서 작성한 cost time 계산 함수를 그대로 사용합니다.
def calculate_average_cost_time(file_path: str) -> Union[float, None]:
    """
    로그 파일에서 첫 5개 에포크의 평균 cost time을 계산합니다.
    """
    cost_times = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip().startswith('Epoch:') and 'cost time:' in line:
                    try:
                        time_str = line.split('cost time:')[1].strip()
                        cost_time = float(time_str)
                        cost_times.append(cost_time)
                        if len(cost_times) == 5:
                            break
                    except (IndexError, ValueError):
                        continue
    except FileNotFoundError:
        print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        return None

    if len(cost_times) < 5:
        # 경고 메시지는 너무 많이 출력될 수 있으므로 주석 처리하거나 필요시 활성화
        # print(f"경고: '{file_path}'에서 5개 에포크를 모두 찾지 못했습니다 (찾은 개수: {len(cost_times)}).")
        return None
    
    return sum(cost_times) / len(cost_times)

def process_logs_to_csv(root_path: str, output_csv_path: str) -> None:
    """
    지정된 경로의 모든 하위 폴더와 로그 파일을 탐색하여 cost time을 추출하고 CSV로 저장합니다.

    Args:
        root_path (str): 로그 폴더들이 있는 최상위 경로.
        output_csv_path (str): 결과를 저장할 CSV 파일 경로.
    """
    all_results: List[Dict[str, Union[str, float]]] = []

    print(f"🔍 '{root_path}' 경로에서 로그 파일 탐색을 시작합니다...")

    # root_path 바로 아래에 있는 모든 항목을 순회 (이것들이 model 폴더가 됨)
    if not os.path.isdir(root_path):
        print(f"오류: '{root_path}'는 유효한 디렉터리가 아닙니다.")
        return

    for model_name in os.listdir(root_path):
        model_path = os.path.join(root_path, model_name)

        # 폴더인 경우에만 처리
        if os.path.isdir(model_path):
            # 해당 모델 폴더 안의 모든 파일을 순회
            for filename in os.listdir(model_path):
                if filename.endswith('.log'):
                    # 파일 이름에서 확장자(.log) 제거
                    base_name = os.path.splitext(filename)[0]
                    
                    # '_'를 기준으로 파일 이름 분리
                    parts = base_name.split('_')
                    if len(parts) == 2:
                        data, pl = parts[0], parts[1]
                        
                        log_file_path = os.path.join(model_path, filename)
                        
                        # cost time 계산
                        cost_time = calculate_average_cost_time(log_file_path)
                        
                        # cost_time이 성공적으로 계산된 경우에만 결과에 추가
                        if cost_time is not None:
                            result_row = {
                                'model': model_name,
                                'data': data,
                                'pl': pl,
                                'cost_time': cost_time
                            }
                            all_results.append(result_row)
    
    if not all_results:
        print("처리할 로그 파일을 찾지 못했습니다. 파일 구조를 확인해주세요.")
        return

    # 결과를 pandas DataFrame으로 변환
    df = pd.DataFrame(all_results)
    
    # DataFrame을 CSV 파일로 저장 (인덱스는 저장하지 않음)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\n✅ 작업 완료! 결과가 '{output_csv_path}' 파일에 저장되었습니다.")
    print("--- 생성된 데이터 미리보기 ---")
    print(df.head())


# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    # ❗ 여기에 실제 로그 파일들이 있는 상위 폴더 경로를 입력하세요.
    # 예: "C:/Users/MyUser/Documents/Experiments/logs"
    root_directory = "/home/Normalizer/Proposed/logs/DDN"  # 테스트를 위한 예시 경로

    # 결과를 저장할 CSV 파일 이름
    output_csv_file = "experiment_cost_times.csv"

    # 메인 함수 실행
    process_logs_to_csv(root_directory, output_csv_file)

🔍 '/home/Normalizer/Proposed/logs/DDN' 경로에서 로그 파일 탐색을 시작합니다...

✅ 작업 완료! 결과가 'experiment_cost_times.csv' 파일에 저장되었습니다.
--- 생성된 데이터 미리보기 ---
     model data   pl   cost_time
0  DLinear  tra  720  111.527186
1  DLinear  elc  720   33.999058
2  DLinear  wea  720   21.497741
3  DLinear  em2  336    9.276879
4  DLinear  eh2  336    1.619305


In [8]:
import re
from typing import Dict, Union

def calculate_epoch_time_sums(file_path: str) -> Union[Dict[str, float], None]:
    """
    로그 파일에서 에포크별 cost time을 구간별로 합산합니다.

    Args:
        file_path (str): 분석할 로그 파일의 경로

    Returns:
        Dict[str, float] | None: 세 가지 합계가 담긴 딕셔너리.
                                 파일을 찾지 못하면 None을 반환합니다.
    """
    # 합계를 저장할 딕셔너리 초기화
    sums = {
        "sum_first_five_epochs": 0.0,
        "sum_after_five_epochs": 0.0,
        "total_sum_all_epochs": 0.0
    }
    
    # Epoch 숫자와 cost time을 한 번에 추출하기 위한 정규표현식
    # (Station, Backbone 접두사를 선택적으로 처리)
    epoch_pattern = re.compile(r"(?:Station |Backbone )?Epoch: (\d+).*cost time: ([\d.]+)")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                match = epoch_pattern.search(line)
                if match:
                    try:
                        epoch_num = int(match.group(1))
                        cost_time = float(match.group(2))
                        
                        # 1. 전체 합계에 우선 더하기
                        sums["total_sum_all_epochs"] += cost_time
                        
                        # 2. 구간별 합계 계산
                        if epoch_num <= 5:
                            sums["sum_first_five_epochs"] += cost_time
                        else:
                            sums["sum_after_five_epochs"] += cost_time
                            
                    except (ValueError, IndexError):
                        # 정규식과 매칭되었으나 숫자 변환에 실패한 경우 (거의 발생하지 않음)
                        continue
                        
    except FileNotFoundError:
        print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        return None

    return sums

# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    # ❗ 여기에 실제 분석하고 싶은 .log 파일의 경로를 입력하세요.
    log_file_path = '/home/Normalizer/Proposed/logs/DDN/iTransformer/elc_336.log'

    # 메인 함수를 실행하고 결과를 받습니다.
    time_sums = calculate_epoch_time_sums(log_file_path)

    # 결과를 출력합니다.
    if time_sums:
        print(f"✅ 로그 파일 '{log_file_path}' 분석 결과:")
        print("-" * 40)
        print(f"  - 1~5 에포크 소요시간 합계  : {time_sums['sum_first_five_epochs']:.4f}초")
        print(f"  - 6 에포크 이후 소요시간 합계 : {time_sums['sum_after_five_epochs']:.4f}초")
        print(f"  - 총 에포크 소요시간 합계    : {time_sums['total_sum_all_epochs']:.4f}초")
        print("-" * 40)

✅ 로그 파일 '/home/Normalizer/Proposed/logs/DDN/iTransformer/elc_336.log' 분석 결과:
----------------------------------------
  - 1~5 에포크 소요시간 합계  : 238.3324초
  - 6 에포크 이후 소요시간 합계 : 916.4257초
  - 총 에포크 소요시간 합계    : 1154.7582초
----------------------------------------


In [1]:
import pandas as pd

df = pd.read_csv('result.csv')
result_list = []
for i in range(len(df)):
    settings = df.Setting[i].split('_')
    dataset = settings[1]
    model_name = settings[4]
    seq_len = settings[7][2:]
    pred_len = settings[9][2:]
    station_lr = settings[-3][3:]
    use_mlp = settings[-2][2:]
    mse = df.MSE[i]
    mae = df.MAE[i]
    
    result_list.append([dataset, model_name, seq_len, pred_len, station_lr, use_mlp, mse, mae])
df_result = pd.DataFrame(result_list, columns=['Dataset', 'Model', 'Seq_Len', 'Pred_Len', 'Station_LR', 'Use_MLP', 'MSE', 'MAE'])
df_result.to_csv('summary_results.csv', index=False)